In [3]:
import pandas as pd
import nltk
from ordered_set import OrderedSet
import numpy
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Word2Vec

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

In [8]:
path="content.csv"

In [9]:
data=pd.read_csv(path)

In [11]:
data.columns

Index(['story_id', 'story_title', 'story_english_title',
       'is_child_created_story', 'stories_status', 'stories_summary',
       'ancestry', 'is_recommended_story', 'reads', 'language_name',
       'organization_name', 'page_type', 'story_derivation_type',
       'story_publishing_type', 'reading_level_cat', 'story_content'],
      dtype='object')

In [13]:
data_text=pd.DataFrame()
data_text['story_content'] = data.loc[:,'story_content']
data_text['index'] = data.loc[:,'story_id']
documents = data_text

In [14]:
type(documents)#.head()

pandas.core.frame.DataFrame

In [19]:
# df.head()

In [20]:
# df.columns

C:\Users\abhishekkumaryadav\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\abhishekkumaryadav\AppData\Roaming\nltk_data.
[nltk_data]     ..
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
stemmer = SnowballStemmer("english")

In [15]:

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [33]:
df=documents
#1314 contained korean
df=df.drop(df.index[1314])
#1311 contained nan value
df=df.drop(df.index[1311])
print(df.shape)

(2046, 2)


In [40]:
#applying preprocess method to entire story_content
processed_docs = list(map(preprocess,df.loc[0:2046,'story_content']))

In [89]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [45]:
#pos function return pos tagging for a list of words  and returns a list
def pos(lis):
    return nltk.pos_tag(lis)

In [46]:
#processed_docs is a list with each element for each story and each element being a list of keywords
processed_docsPOS=list(map(pos,processed_docs))    

In [51]:
#separating noun and adjective pos tags from this list of tuples
l1=['NN','NNS','NNP','NNPS']
l2=['JJ','JJR','JJS']
def sep(lis):
    ansNoun=[]
    ansAdj=[]
    for i in lis:
        if(i[1] in l1):
            ansNoun.append(i[0])
        if(i[1] in l2):
            ansAdj.append(i[0])
    return ansNoun,ansAdj
        

In [52]:
processed_docsPOSSep=list(map(sep,processed_docsPOS))

In [53]:
#Separating the Noun And Adjective keywords separately
processed_docsPOSNoun=[]
processed_docsPOSAdj=[]
for i in processed_docsPOSSep:
    processed_docsPOSNoun.append(i[0])
    processed_docsPOSAdj.append(i[1])

In [ ]:
#doc2vec  gives a vector for a document
#word2vec gives a vector for a word
#calculating the similarity score b/w word and document

In [64]:
#here processed_docsPOSSep replace with the file downloaded from standford for training the model
model1 = Word2Vec(processed_docsPOSAdj, min_count=1,size=50)
model2 = Word2Vec(processed_docsPOSNoun, min_count=1,size=50)

In [69]:
#working with doc2vec



In [70]:
documents1 = [TaggedDocument(doc, [i]) for i, doc in enumerate(processed_docsPOSAdj)]
model11 = Doc2Vec(documents1, vector_size=50, window=2, min_count=1, workers=4)
documents2 = [TaggedDocument(doc, [i]) for i, doc in enumerate(processed_docsPOSNoun)]
model12 = Doc2Vec(documents2, vector_size=50, window=2, min_count=1, workers=4)

In [ ]:
#Now we will apply MMD equation  S=λA- (1-λ) max sim(Wi,Wj)

In [ ]:
# We fixed λ
# to 0.5 in the adapted MMR equation (2), to ensure
# equal importance to informativeness and diversity.

In [74]:
#model1 is model for doc2vec
#model is model for word2vec
STAdj=[]
for i in processed_docsPOSAdj:
    #vector for this document
    vectorD=model11.infer_vector(i)
    S=[]
    for j in i:
        #vector for this word
        vectorW=model1.wv[j]
        A=numpy.dot(vectorD,vectorW)
#         maxSim=0
#         for k in i:
#             if(numpy.dot(model.wv[j],model.wv[k])>maxSim):
#                 maxSim=numpy.dot(model.wv[j],model.wv[k])
#         S.append(0.5*A-0.5*maxSim)
        S.append((A,j))
    STAdj.append(S)

In [75]:
#model1 is model for doc2vec
#model is model for word2vec
STNoun=[]
for i in processed_docsPOSNoun:
    #vector for this document
    vectorD=model12.infer_vector(i)
    S=[]
    for j in i:
        #vector for this word
        vectorW=model2.wv[j]
        A=numpy.dot(vectorD,vectorW)
#         maxSim=0
#         for k in i:
#             if(numpy.dot(model.wv[j],model.wv[k])>maxSim):
#                 maxSim=numpy.dot(model.wv[j],model.wv[k])
#         S.append(0.5*A-0.5*maxSim)
        S.append((A,j))
    STNoun.append(S)

In [76]:
def getKey(item):
    return item[0]

In [84]:
#sorting the keywords as per their similarity score
STAdj1=[]
for i in STAdj:
    STAdj1.append(sorted(i,key=getKey))

In [85]:
STNoun1=[]
for i in STNoun:
    STNoun1.append(sorted(i,key=getKey))

In [98]:
#replacing the duplicates with just one of it's own kind
STAdj11=[]
for i in STAdj1:
    STAdj11.append(list(OrderedSet(i)))

In [99]:
STNoun11=[]
for i in STNoun1:
    STNoun11.append(list(OrderedSet(i)))

In [100]:
#limiting the words in each document to 10 
STAdj2=[]
for i in STAdj11:
    count=0
    ST3=[]
    for j in i:
        if(count<=10):
            ST3.append(j[1])
        else:
            break
        count+=1
    STAdj2.append(ST3)

In [101]:
STNoun2=[]
for i in STNoun11:
    count=0
    ST3=[]
    for j in i:
        if(count<=10):
            ST3.append(j[1])
        else:
            break
        count+=1
    STNoun2.append(ST3)

In [103]:
#converting 2-D list to dataFrame
#final result are these two csv files each under category NOUN and ADJECTIVE
df =pd.DataFrame(STAdj2)
df.to_csv("STAdj.csv")

In [104]:
df =pd.DataFrame(STNoun2)
df.to_csv("STNoun.csv")

In [ ]:
#summa keywords.keywords are not performing well as evident from the first story content output

In [58]:
a1=remove_special_characters(w)
print(a1)
a=keywords.keywords(a1)
print(a)
y=[s.strip() for s in a.splitlines()]
print(y)

A fawn was racing in the forest He was ahead of the rabbit He was ahead of the elephant He leapt and cleared the stream He ran past the crumbling wall There was a large boulder on the grassy plain He stumbled and fell down He burst into tears The monkey massaged his leg Tears flowed from the fawns eyes Brother Bear picked him up The fawn didnt stop crying His mother came She said Look well beat up this bad boulder The fawn said Oh dont do that or he will also start crying His mother laughed So did the fawn
fawn
fawns eyes
didnt
oh dont
bear
['fawn', 'fawns eyes', 'didnt', 'oh dont', 'bear']


In [ ]:
###############################################################################

In [ ]:
#from the previous code

In [39]:
x=df["story_content"]

In [41]:
# (df["story_content"][1314])

In [42]:
import re
def remove_special_characters(text, remove_digits=False):
    
    #checking that should we remove the digits as well or not
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    
    #find the pattern in text and replace it with ''
    text = re.sub(pattern,'', text)
    return text

In [ ]:
x1=[]
for item in range(len(x)):
    if(item == 1314):
        continue
    a1=remove_special_characters(x[item])
    a=keywords.keywords(a1)
    y=[s.strip() for s in a.splitlines()]
    x1.append(y)

In [54]:
def add_keyword(x,i,f):
    k=[]
    for item in x[i:f]:
        print("##########1")
        a1=remove_special_characters(item)
        print(a1)
        a=keywords.keywords(a1)
        print("##########2")
        print(a)
        y=[s.strip() for s in a.splitlines()]
        print("##########3")
        print(y)
        k.append(y)
        print("##########4")
        print(k)
    return k

In [56]:
#since this didn't work
# from summa import commons, graph, keywords, pagerank_weighted
# from summa import summarizer, syntactic_unit, textrank

a=add_keyword(x,0,1)

##########1
A fawn was racing in the forest He was ahead of the rabbit He was ahead of the elephant He leapt and cleared the stream He ran past the crumbling wall There was a large boulder on the grassy plain He stumbled and fell down He burst into tears The monkey massaged his leg Tears flowed from the fawns eyes Brother Bear picked him up The fawn didnt stop crying His mother came She said Look well beat up this bad boulder The fawn said Oh dont do that or he will also start crying His mother laughed So did the fawn
##########2
fawn
fawns eyes
didnt
oh dont
bear
##########3
['fawn', 'fawns eyes', 'didnt', 'oh dont', 'bear']
##########4
[['fawn', 'fawns eyes', 'didnt', 'oh dont', 'bear']]


In [57]:
a

[['fawn', 'fawns eyes', 'didnt', 'oh dont', 'bear']]

In [ ]:
#we tried this
x1=[]
for item in x[0:1000]:
    a1=remove_special_characters(item)
    a=keywords.keywords(a1)
    y=[s.strip() for s in a.splitlines()]
    x1.append(y)

In [ ]:
#it's definitely between 1300-1350
#1400-1500 is safe
#1500-1700 is safe
#1700 to 1900 is safe